### Read the script

In [1]:
import os
import mne
from ieeg.navigate import crop_empty_data
from ieeg.io import get_data, raw_from_layout, save_derivative

In [2]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

In [3]:
subject = subjects[7]
# subject_Tag = 'D107B'
# subject_Tag = 'D0070'

In [ ]:
raw = raw_from_layout(layout, subject=subject, preload=True, extension='.edf')
print(subjects)

### Linear noise filtering
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_clean.html

In [4]:
from ieeg.mt_filter import line_filter

In [ ]:
# n_jobs=-1 means take all the processors, n_jobs=-2 means take all - 1 processors
# Keep all the other parameters not changed
# %% filter data
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='700ms', freqs=[60], notch_widths=20)
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='20s', freqs=[60, 120, 180, 240],
            notch_widths=20)

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!! This should be removed in the future. We don't need to have to save the line noise filtered files. Save them after bad channels removal!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Check if derivatives folder exists and create if not
bids_root='C:\\Users\\bl314\\Box\\CoganLab\\BIDS-1.0_LexicalDecRepDelay\\BIDS'
if not os.path.exists(os.path.join(bids_root, "derivatives")):
    os.mkdir(os.path.join(bids_root, "derivatives"))
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
elif not os.path.exists(os.path.join(bids_root, "derivatives", "clean")):
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
save_derivative(raw, layout, "clean", True)

## Remove eeg channels
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms.html
### <span style="color:red">Please pay attention to the section above and see what channels do not have locations when the raw was loaded. Write them down and remove them. They are usually EEG channels</span>

In [5]:
# !!!!!!!!!!!!!!!!!!!!!!!! This should be removed in the future. We don't need to have to save the line noise filtered files. Save them after bad channels removal!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
raw = raw_from_layout(layout.derivatives['derivatives/clean'], subject=subject, desc='clean',extension='.edf',preload=False)

Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\s

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_acq-01_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 424 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_acq-02_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 424 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\clean\sub-D0096\ieeg\sub-D0096_acq-02_space-ACPC_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: Omitted 422 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


In [6]:
# Remove EEG channels for D101
found=1
if subject=='D0053':
    eeg_channels_to_exclude=[]
elif subject=='D0054':
    eeg_channels_to_exclude=[]
elif subject=='D0055':
    eeg_channels_to_exclude=[]
elif subject=='D0070':
    eeg_channels_to_exclude=[]
elif subject=='D0094':
    eeg_channels_to_exclude=[]
elif subject=='D0096':
    eeg_channels_to_exclude=[]
elif subject=='D0101':
    eeg_channels_to_exclude = ['Fp1', 'Fp2', 'F3','F4','C3','C4','P3','P4','F7','F8','T3','T4','T5','T6','O1','O2','Fz','Cz','Pz']
elif subject=='D0102':
    eeg_channels_to_exclude = ['T5', 'T6', 'FZ', 'CZ', 'PZ', 'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', '02', 'F7', 'F8', 'T3', 'T4']
elif subject=='D0103':
    eeg_channels_to_exclude = ['FZ', 'CZ', 'PZ', 'F7', 'F8', 'T5', 'T6', 'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'T3', 'T4']
elif subject=='D0107':
    eeg_channels_to_exclude=[]
else:
    print("Subject not found, please check.")
    found=0
    
if found and eeg_channels_to_exclude:
    raw.drop_channels(eeg_channels_to_exclude)
#raw.load_data()

## Remove outlier channels

In [7]:
from ieeg.navigate import channel_outlier_marker
# mark channel outliers as bad
rawsmall = crop_empty_data(raw)
rawsmall.info['bads'] = channel_outlier_marker(raw, 3, 2)
# Exclude bad channels
raw.drop_channels(rawsmall.info['bads'])
del rawsmall
#raw.load_data()

outlier round 1 channels: ['LTAM15']
outlier round 2 channels: ['LTAM15', 'LTMM8']
outlier round 2 channels: ['LTAM15', 'LTMM8', 'LIP18']


In [8]:
# Check if derivatives folder exists and create if not
bids_root='C:\\Users\\bl314\\Box\\CoganLab\\BIDS-1.0_LexicalDecRepDelay\\BIDS'
if not os.path.exists(os.path.join(bids_root, "derivatives")):
    os.mkdir(os.path.join(bids_root, "derivatives"))
    os.mkdir(os.path.join(bids_root, "derivatives", "a"))
elif not os.path.exists(os.path.join(bids_root, "derivatives", "a")):
    os.mkdir(os.path.join(bids_root, "derivatives", "a"))
save_derivative(raw, layout, "a", True)

Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.tsv'...
Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.json'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-01_space-ACPC_electrodes.tsv'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-01_space-ACPC_coordsystem.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify custom event codes, please pass "event_id".
Used Annotations descriptions: [np.str_('Auditory_stim/Repeat/Nonword/banic/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/delin/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/galel/CORRECT'), np.str_('Auditory_stim/Repeat/Nonword/gapel/CORRECT'), np.str_('Auditor

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:242: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(run, bids_path, allow_preload=True, format=format,
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:242: RuntimeWarning: EDF format requires equal-length data blocks, so 1 seconds of edge values were appended to all channels when writing the final block.
  write_raw_bids(run, bids_path, allow_preload=True, format=format,


Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv'...
Wrote C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv entry with ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-01_desc-a_ieeg.edf.
Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.tsv'...
Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.json'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-01_space-ACPC_electrodes.tsv'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-01_space-ACPC_coordsystem.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify cu

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:242: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(run, bids_path, allow_preload=True, format=format,
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:242: RuntimeWarning: EDF format requires equal-length data blocks, so 1 seconds of edge values were appended to all channels when writing the final block.
  write_raw_bids(run, bids_path, allow_preload=True, format=format,


Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv'...
Wrote C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv entry with ieeg\sub-D0096_task-LexicalDecRepDelay_acq-01_run-02_desc-a_ieeg.edf.
Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.tsv'...
Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.json'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-02_space-ACPC_electrodes.tsv'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-02_space-ACPC_coordsystem.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify cu

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:242: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(run, bids_path, allow_preload=True, format=format,
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:242: RuntimeWarning: EDF format requires equal-length data blocks, so 1 seconds of edge values were appended to all channels when writing the final block.
  write_raw_bids(run, bids_path, allow_preload=True, format=format,


Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv'...
Wrote C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv entry with ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-03_desc-a_ieeg.edf.
Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.tsv'...
Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\participants.json'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-02_space-ACPC_electrodes.tsv'...
Writing 'C:/Users/bl314/Box/CoganLab/BIDS-1.0_LexicalDecRepDelay/BIDS/derivatives/a/sub-D0096/ieeg/sub-D0096_acq-02_space-ACPC_coordsystem.json'...
The provided raw data contains annotations, but you did not pass an "event_id" mapping from annotation descriptions to event codes. We will generate arbitrary event codes. To specify cu

C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:242: RuntimeWarning: Converting data files to EDF format
  write_raw_bids(run, bids_path, allow_preload=True, format=format,


Writing 'C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv'...
Wrote C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\derivatives\a\sub-D0096\sub-D0096_scans.tsv entry with ieeg\sub-D0096_task-LexicalDecRepDelay_acq-02_run-04_desc-a_ieeg.edf.
